In [1]:
import sys,os
sys.path.append("../")
from utils import *
from model import *
from loss import *
from dataset import *
from torchvision import transforms
import torch.nn as nn
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
import time
root_dir = '/home/harsh/Downloads/ModelNet10/ModelNet10/'

In [2]:
bs = 64
epochs = 100
lr = 0.001
alpha = 0.0001
train_tfms = transforms.Compose([PointSampler(1024),TrainTransforms()])
test_tfms = transforms.Compose([PointSampler(1024),TestTransforms()])
train_ds = ModelNet10(root_dir = root_dir,folder='train',transforms=train_tfms)
test_ds = ModelNet10(root_dir = root_dir,folder = 'test',transforms=test_tfms)

train_dl = DataLoader(train_ds,batch_size = bs,shuffle = True)
test_dl = DataLoader(test_ds,batch_size = bs,shuffle = True)

model = PointNet(classes = len(train_ds.classes)).cuda()

optim = torch.optim.Adam(model.parameters(),lr = 0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optim,step_size=20,gamma=0.5)
criterion = PointNetLoss(alpha=alpha)

writer = SummaryWriter('runs/experiment_1')

In [3]:
len(train_ds)

3991

In [4]:
len(test_ds)

908

In [5]:
len(train_ds.classes)

10

In [ ]:
for epoch in range(epochs):
    model.train()
    av_tr_loss = 0
    train_acc = 0
    tic = time.time()
    
    for i,(pointclouds,labels) in enumerate(train_dl):
        optim.zero_grad()
        pointclouds = pointclouds.type(torch.FloatTensor)
        outputs,m3,m64 = model(pointclouds.cuda())
        loss = criterion(outputs,labels.cuda(),m3,m64)
        av_tr_loss += loss.item()
        acc = torch.sum(torch.argmax(outputs.cpu(),dim = 1) == labels)
        train_acc += acc
        loss.backward()
        optim.step()
        scheduler.step()
        toc = time.time()
        print(toc-tic)
#     if (epoch+1)%5 == 0:
#         model.eval()
#         av_test_loss = 0
#         test_acc = 0
#         with torch.no_grad():
#             for i,(pointclouds,labels) in enumerate(test_dl):
#                 pointclouds = pointclouds.type(torch.FloatTensor)
#                 outputs,m3,m64 = model(pointclouds.cuda())
#                 loss_test = criterion(outputs,labels.cuda(),m3,m64)
#                 av_test_loss += loss_test.item()
#                 test_acc += torch.sum(torch.argmax(outputs.cpu(),dim = 1) == labels.cuda())

#         print("Epoch [{}/{}] --> Testing Loss:{}, Testing Accuracy:{}".format(epoch,epochs,av_test_loss/len(test_ds), test_acc/len(test_ds)))
#         writer.add_scalar('Test Loss',av_test_loss/len(test_ds),epoch//5 + 1)
#         writer.add_scalar('Test acc',test_acc/len(test_ds),epoch//5 + 1)
#     writer.add_scalar('Train Loss',av_tr_loss/len(train_ds),epoch)
#     writer.add_scalar('Train acc',train_acc/len(train_ds),epoch)
    
#     print("Epoch [{}/{}] --> Training Loss:{:.2f}, Testing Accuracy:{:.2f}".format(epoch,epochs,av_tr_loss/len(train_ds), train_acc/len(train_ds)))
# writer.flush()

14.290147304534912
26.005099534988403
37.39424753189087
47.57975745201111
66.77631998062134
76.27570962905884
89.76783919334412
102.06793427467346
114.94538903236389
132.9121754169464
142.79286360740662
159.13141894340515
171.93074584007263
185.2412657737732
193.21211695671082
207.36930203437805
220.70982003211975
241.55277729034424
261.37028551101685
275.3287544250488
292.59160590171814
307.9662289619446
320.85010838508606
336.19637274742126
349.2273952960968
361.66480922698975
374.90638613700867
